## Creating a smart model with feature engineering and tuning

In [1]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import dump
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import os

In [2]:
df = pd.read_csv("../../data/car_price_prediction.csv")
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


### Data Cleaning and Feature Engineering
Going through each column to clean and create new features when necessary

In [3]:
# viewing all of the columns and their types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

The ID column probably won't contain any valuable information for the model, but we can use it to see if there are any duplicate entries in the dataframe and remove them if there are

In [4]:
# viewing the number of times each ID shows up in the dataframe
df["ID"].value_counts()

ID
45815365    8
45815361    8
45815363    7
45815368    7
45723475    7
           ..
45774312    1
45732621    1
45773011    1
45774019    1
45813273    1
Name: count, Length: 18924, dtype: int64

There are actually quite a few duplicate rows. They need to be removed.

In [5]:
# Removing duplicate rows
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
df.drop_duplicates(subset=["ID"], keep="first", inplace=True)

print(df["ID"].value_counts())

# dropping ID column
df.drop(columns=["ID"], inplace=True)
df.head()

ID
45654403    1
45802973    1
45769757    1
45792992    1
45776193    1
           ..
45659914    1
45728551    1
45758183    1
45797245    1
45813273    1
Name: count, Length: 18924, dtype: int64


,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [6]:
# viewing the smallest price cars
df.sort_values(by="Price", ascending=True).head(10)

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
7815,1,-,OPEL,Astra,1999,Sedan,Yes,Petrol,1.6,122231 km,4.0,Manual,Front,04-May,Left wheel,Silver,4
16992,1,1202,CHEVROLET,Lacetti,2006,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,2
753,3,-,NISSAN,X-Terra,2004,Jeep,No,Petrol,3.3,323200 km,6.0,Automatic,4x4,04-May,Left wheel,Black,2
15347,3,-,BMW,525,1995,Sedan,No,Petrol,2.8,2147483647 km,6.0,Manual,Rear,04-May,Left wheel,Black,3
9730,3,900,KIA,Sportage,2015,Jeep,No,Petrol,2.4,50345 km,4.0,Tiptronic,4x4,04-May,Left wheel,Blue,12
10885,3,-,CHEVROLET,Cruze,2018,Sedan,No,Petrol,1.4,61000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,8
4776,3,810,VOLKSWAGEN,Jetta,2016,Sedan,Yes,Petrol,1.8 Turbo,41000 km,4.0,Automatic,Front,04-May,Left wheel,White,8
17596,3,-,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,106422 km,4.0,Tiptronic,Front,04-May,Left wheel,White,6
5890,3,-,MERCEDES-BENZ,CLK 230,2004,Coupe,Yes,Petrol,2.3,328000 km,4.0,Manual,Rear,02-Mar,Left wheel,Blue,0
11636,3,-,MERCEDES-BENZ,C 220,1998,Sedan,No,Diesel,2.2,20600 km,4.0,Manual,Rear,04-May,Left wheel,Black,2


There are some extremely low prices in this dataset. This is not realistic. I will remove every car that costs less than $500.

In [7]:
# dropping rows where price is less than 500
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
df.drop(df[df["Price"] < 500].index, inplace=True)

# viewing the smallest price cars
df.sort_values(by="Price", ascending=True).head()

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
19194,500,-,FORD,Transit,1998,Microbus,No,Diesel,2.5,760000 km,4.0,Manual,Rear,04-May,Left wheel,White,2
13706,500,-,NISSAN,Tiida,2007,Hatchback,Yes,Petrol,1.8,0 km,4.0,Variator,Front,04-May,Right-hand drive,Grey,6
15334,549,831,FORD,Fiesta,2017,Hatchback,Yes,Petrol,1.6,96088 km,4.0,Automatic,Front,04-May,Left wheel,White,12
2393,549,1017,HYUNDAI,Elantra,2017,Sedan,Yes,Petrol,2,87357 km,4.0,Automatic,Front,04-May,Left wheel,Blue,12
8124,549,915,AUDI,100,2014,Jeep,Yes,Petrol,3,143392 km,6.0,Automatic,4x4,04-May,Left wheel,Black,12


Now I will do the same with the cars that are the most expensive.

In [8]:
df.sort_values(by="Price", ascending=False).head()

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
16983,26307500,-,OPEL,Combo,1999,Goods wagon,No,Diesel,1.7,99999 km,4.0,Manual,Front,02-Mar,Left wheel,Blue,0
8541,872946,2067,LAMBORGHINI,Urus,2019,Universal,Yes,Petrol,4,2531 km,8.0,Tiptronic,4x4,04-May,Left wheel,Black,0
1225,627220,-,MERCEDES-BENZ,G 65 AMG 63AMG,2020,Jeep,Yes,Petrol,6.3 Turbo,0 km,8.0,Tiptronic,4x4,04-May,Left wheel,Black,12
5008,308906,1694,PORSCHE,911,2016,Coupe,Yes,Petrol,4,8690 km,6.0,Automatic,Rear,02-Mar,Left wheel,Black,12
9367,297930,-,MERCEDES-BENZ,AMG GT S,2015,Coupe,Yes,Petrol,4.0 Turbo,20000 km,8.0,Tiptronic,Rear,02-Mar,Left wheel,White,16


$26 million seems excessive for this dataset, so it will removed

In [9]:
# dropping rows where the price is over $1 million
df.drop(df[df["Price"] > 1000000].index, inplace=True)

In [10]:
# viewing Levy value counts
df["Levy"].value_counts()

Levy
-       5346
765      471
891      447
639      395
640      366
        ... 
3156       1
2908       1
1719       1
4283       1
1045       1
Name: count, Length: 545, dtype: int64

There are a large amount of columns where the Levy is null. We will turn these into 0.

In [11]:
# Replacing the Levy column with 0 if there is "-"
df["Levy"] = df["Levy"].map(lambda x: 0 if x == "-" else x).astype("int64")
df["Levy"].value_counts()

Levy
0       5346
765      471
891      447
639      395
640      366
        ... 
3156       1
2908       1
1719       1
4283       1
1045       1
Name: count, Length: 545, dtype: int64

In [12]:
# viewing the amounts of Manufacturers
list(df["Manufacturer"].value_counts().items())

[('HYUNDAI', 3589),
 ('TOYOTA', 3150),
 ('MERCEDES-BENZ', 1785),
 ('CHEVROLET', 985),
 ('FORD', 972),
 ('BMW', 971),
 ('HONDA', 844),
 ('LEXUS', 756),
 ('NISSAN', 577),
 ('VOLKSWAGEN', 550),
 ('SSANGYONG', 439),
 ('KIA', 395),
 ('OPEL', 391),
 ('MITSUBISHI', 278),
 ('SUBARU', 251),
 ('AUDI', 230),
 ('MAZDA', 155),
 ('JEEP', 132),
 ('DAEWOO', 91),
 ('DODGE', 85),
 ('SUZUKI', 73),
 ('FIAT', 70),
 ('PORSCHE', 49),
 ('LAND ROVER', 48),
 ('VAZ', 47),
 ('MINI', 45),
 ('JAGUAR', 37),
 ('RENAULT', 36),
 ('INFINITI', 24),
 ('CHRYSLER', 23),
 ('SKODA', 20),
 ('VOLVO', 17),
 ('PEUGEOT', 17),
 ('BUICK', 14),
 ('GMC', 13),
 ('DAIHATSU', 13),
 ('UAZ', 12),
 ('CADILLAC', 12),
 ('ACURA', 11),
 ('LINCOLN', 11),
 ('GAZ', 11),
 ('CITROEN', 9),
 ('SCION', 7),
 ('ALFA ROMEO', 4),
 ('HUMMER', 4),
 ('ISUZU', 4),
 ('MERCURY', 4),
 ('MASERATI', 3),
 ('MOSKVICH', 3),
 ('ROVER', 3),
 ('BENTLEY', 2),
 ('SAAB', 2),
 ('FERRARI', 2),
 ('SEAT', 2),
 ('ZAZ', 2),
 ('სხვა', 2),
 ('LANCIA', 1),
 ('TESLA', 1),
 ('HAVAL', 

64 columns of manufacturers is a bit excessive, but it is important information. To reduce the number of columns, I will replace any manufacturer with less than 45 entries with "Other".

In [13]:
# save manufacturer value counts as a dictionary
manufacturer_counts = dict(df["Manufacturer"].value_counts())

# changing the Manufacturer column to "Other" for any entry with a manufacturer that has less than 45 entries
df["Manufacturer"] = df["Manufacturer"].map(lambda x: x if manufacturer_counts[x] > 45 else "Other")

print(len(df["Manufacturer"].value_counts()))
df["Manufacturer"].value_counts()

26


Manufacturer
HYUNDAI          3589
TOYOTA           3150
MERCEDES-BENZ    1785
CHEVROLET         985
FORD              972
BMW               971
HONDA             844
LEXUS             756
NISSAN            577
VOLKSWAGEN        550
SSANGYONG         439
KIA               395
OPEL              391
Other             377
MITSUBISHI        278
SUBARU            251
AUDI              230
MAZDA             155
JEEP              132
DAEWOO             91
DODGE              85
SUZUKI             73
FIAT               70
PORSCHE            49
LAND ROVER         48
VAZ                47
Name: count, dtype: int64

26 is still a lot, but definitely more manageable. Considering that manufacturer is critical information, I am okay with it.

In [14]:
print(len(df["Model"].value_counts()))
df["Model"].value_counts()

1567


Model
Sonata                    987
Prius                     901
Elantra                   871
Camry                     784
Santa FE                  522
                         ... 
Musa                        1
1500,1600 Schtufenheck      1
E 420                       1
C 250 luxury                1
Prius C aqua                1
Name: count, Length: 1567, dtype: int64

1567 is way too many. I think car model is too specific of information. I am just going to remove this column

In [15]:
df.drop(columns=["Model"], inplace=True)

In [16]:
df["Prod. year"].value_counts()

Prod. year
2014    1929
2012    1910
2013    1729
2015    1420
2016    1394
2011    1370
2010    1315
2017     817
2008     630
2009     559
2018     460
2007     426
2005     353
2004     332
2003     331
2006     286
2002     270
2000     270
2019     254
2001     245
1998     203
1999     202
1997     149
1996     108
1995     101
2020      44
1994      41
1992      30
1993      22
1990      18
1988      12
1991      10
1989       6
1986       6
1985       5
1987       5
1984       5
1953       3
1983       3
1964       2
1977       2
1978       2
1965       2
1980       2
1974       1
1943       1
1957       1
1968       1
1982       1
1981       1
1973       1
Name: count, dtype: int64

Production year looks fine, so I will leave it alone.

In [17]:
df["Category"].value_counts()

Category
Sedan          7867
Jeep           4855
Hatchback      2489
Minivan         611
Coupe           506
Universal       361
Microbus        290
Goods wagon     225
Pickup           47
Cabriolet        33
Limousine         6
Name: count, dtype: int64

There are 11 categories. I think this is okay, so I will leave this column alone as well.

In [18]:
df["Leather interior"].value_counts()

Leather interior
Yes    12348
No      4942
Name: count, dtype: int64

Leather interior needs to be converted to a Boolean.

In [19]:
# converting Leather interior to 0 if No, 1 if Yes
df["Leather interior"] = df["Leather interior"].map(lambda x: 0 if x == "No" else 1)
df["Leather interior"].value_counts()

Leather interior
1    12348
0     4942
Name: count, dtype: int64

In [20]:
df["Fuel type"].value_counts()

Fuel type
Petrol            9135
Diesel            3814
Hybrid            2926
LPG                868
CNG                461
Plug-in Hybrid      85
Hydrogen             1
Name: count, dtype: int64

I am going to combine plug-in hybrid with hybrid and remove the hydrogen car.

In [21]:
# drop the row with the Hydrogen car
df.drop(df[df["Fuel type"] == "Hydrogen"].index, inplace=True)

# convert the rows with "Plug-in Hybrid" to Hybrid
df["Fuel type"] = df["Fuel type"].map(lambda x: "Hybrid" if x == "Plug-in Hybrid" else x)

df["Fuel type"].value_counts()

Fuel type
Petrol    9135
Diesel    3814
Hybrid    3011
LPG        868
CNG        461
Name: count, dtype: int64

The Engine volume column is a string for some reason. I need to investigate why.

In [22]:
list(df["Engine volume"].value_counts().items())

[('2', 3648),
 ('2.5', 2024),
 ('1.8', 1556),
 ('1.6', 1397),
 ('1.5', 1174),
 ('3.5', 979),
 ('2.4', 848),
 ('3', 720),
 ('1.3', 421),
 ('2.0 Turbo', 403),
 ('1.4', 319),
 ('4.6', 310),
 ('3.0 Turbo', 257),
 ('2.2', 221),
 ('2.2 Turbo', 210),
 ('1.8 Turbo', 170),
 ('3.3', 164),
 ('1.4 Turbo', 154),
 ('1.7', 149),
 ('3.2', 143),
 ('2.7', 121),
 ('1.6 Turbo', 116),
 ('1', 115),
 ('3.6', 105),
 ('2.3', 105),
 ('1.2', 100),
 ('4', 94),
 ('2.5 Turbo', 90),
 ('4.7', 81),
 ('3.7', 67),
 ('4.4', 67),
 ('5', 64),
 ('1.5 Turbo', 63),
 ('2.8', 57),
 ('5.5', 48),
 ('2.4 Turbo', 48),
 ('2.7 Turbo', 46),
 ('5.7', 38),
 ('1.9', 38),
 ('1.7 Turbo', 38),
 ('3.5 Turbo', 35),
 ('3.8', 32),
 ('4.8', 32),
 ('1.9 Turbo', 31),
 ('4.4 Turbo', 24),
 ('3.2 Turbo', 23),
 ('0.7', 19),
 ('1.3 Turbo', 18),
 ('2.6', 18),
 ('4.7 Turbo', 18),
 ('4.2', 16),
 ('2.3 Turbo', 16),
 ('2.8 Turbo', 12),
 ('4.3', 12),
 ('4.5', 12),
 ('2.9 Turbo', 12),
 ('5.5 Turbo', 11),
 ('2.9', 9),
 ('6.2', 8),
 ('0.8', 8),
 ('0.2', 8),
 ('

It looks like some of the entries in this column contain the word "Turbo" at the end of them. To fix this, I will make "Turbo" a new Boolean column and remove the word "Turbo" from the Fuel Type column and convert it to a float.

In [23]:
# add a new "Turbo" column based. This is done by performing a split on whitespace for the values in the Engine volume column
# If the length of the split is 2, then Turbo = 1, else 0
df["Turbo"] = df["Engine volume"].map(lambda x: 1 if len(x.split(" ")) == 2 else 0)

# removing the word "Turbo" from the Engine volume column
df["Engine volume"] = df["Engine volume"].map(lambda x: x.split(" ")[0]).astype("float64")

df["Engine volume"].describe()

count    17289.000000
mean         2.286043
std          0.858194
min          0.000000
25%          1.800000
50%          2.000000
75%          2.500000
max         20.000000
Name: Engine volume, dtype: float64

The Mileage column needs to be converted to integers. Every entry has "km" at the end of it. This needs to be removed.

In [24]:
df["Mileage"] = df["Mileage"].map(lambda x: x.split(" ")[0]).astype("int64")
df["Mileage"].describe()

count    1.728900e+04
mean     1.497713e+06
std      4.763519e+07
min      0.000000e+00
25%      7.000500e+04
50%      1.249120e+05
75%      1.853340e+05
max      2.147484e+09
Name: Mileage, dtype: float64

In [25]:
print(df["Mileage"].sort_values(ascending=True).head())
print(df["Mileage"].sort_values(ascending=False).head(10))

17115    0
4711     0
12507    0
8743     0
3615     0
Name: Mileage, dtype: int64
12734    2147483647
15393    2147483647
19167    2147483647
11901    2147483647
6157     2147483647
2278     2147483647
17582    1777777778
7724     1234567899
19199    1111111111
12904     999999999
Name: Mileage, dtype: int64


There appears to be a few cars with insanely large Mileage values. These need to be removed. I will choose 450,000 km as a generous cutoff value. Based on a few Google searches, this seems to be around unrealistic territory without removing too many rows.

In [26]:
# removing rows where mileage is greater than 450000
df.drop(df[df["Mileage"] > 450000].index, inplace=True)
print(df["Mileage"].sort_values(ascending=True).head())
print(df["Mileage"].sort_values(ascending=False).head())

9574     0
1097     0
2997     0
14002    0
5529     0
Name: Mileage, dtype: int64
7549     450000
1148     450000
10171    448982
10683    448982
2319     448982
Name: Mileage, dtype: int64


In [27]:
df["Cylinders"].value_counts()

Cylinders
4.0     12825
6.0      2926
8.0       832
5.0       153
3.0       105
2.0        38
12.0       35
1.0        32
10.0       11
16.0        5
7.0         2
9.0         1
14.0        1
Name: count, dtype: int64

The Cylinders column looks fine.

In [28]:
df["Gear box type"].value_counts()

Gear box type
Automatic    11556
Tiptronic     2971
Manual        1745
Variator       694
Name: count, dtype: int64

Gear box type looks fine as well

In [29]:
df["Drive wheels"].value_counts()

Drive wheels
Front    11514
4x4       3398
Rear      2054
Name: count, dtype: int64

Drive wheels looks good, too.

In [30]:
df["Wheel"].value_counts()

Wheel
Left wheel          15667
Right-hand drive     1299
Name: count, dtype: int64

I don't think that the car driving on the left side of the road or the right side of the road would affect the price in a major way. I am going to remove this column.

In [31]:
# removing "Wheel" column
df.drop(columns=["Wheel"], inplace=True)

Dropping the Doors column as well

In [32]:
df.drop(columns=["Doors"], inplace=True)

In [33]:
df["Color"].value_counts()

Color
Black            4435
White            3926
Silver           3386
Grey             2098
Blue             1260
Red               577
Green             290
Carnelian red     168
Brown             161
Golden            140
Beige             126
Orange            119
Sky blue          115
Yellow            104
Purple             37
Pink               24
Name: count, dtype: int64

There are quite a lot of colors. I am going to group every color with less than 1,000 entries as "Other".

In [34]:
# creating dictionary of value counts of car colors
color_counts = dict(df["Color"].value_counts())

# converting "Color" to "Other" for rows with a color that appears less than 1000 time in the dataframe
df["Color"] = df["Color"].map(lambda x: x if color_counts[x] > 1000 else "Other")

df["Color"].value_counts()

Color
Black     4435
White     3926
Silver    3386
Grey      2098
Other     1861
Blue      1260
Name: count, dtype: int64

In [35]:
df["Airbags"].describe()

count    16966.000000
mean         6.498055
std          4.215080
min          0.000000
25%          4.000000
50%          5.000000
75%         12.000000
max         16.000000
Name: Airbags, dtype: float64

Airbags look fine.

Alright, the dataframe should be clean and ready to have the categorical columns be encoded.

In [36]:
df.head()

,Price,Levy,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Color,Airbags,Turbo
0,13328,1399,LEXUS,2010,Jeep,1,Hybrid,3.5,186005,6.0,Automatic,4x4,Silver,12,0
1,16621,1018,CHEVROLET,2011,Jeep,0,Petrol,3.0,192000,6.0,Tiptronic,4x4,Black,8,0
2,8467,0,HONDA,2006,Hatchback,0,Petrol,1.3,200000,4.0,Variator,Front,Black,2,0
3,3607,862,FORD,2011,Jeep,1,Hybrid,2.5,168966,4.0,Automatic,4x4,White,0,0
4,11726,446,HONDA,2014,Hatchback,1,Petrol,1.3,91901,4.0,Automatic,Front,Silver,4,0


In [37]:
df_dummy = pd.get_dummies(df, columns=["Manufacturer", "Category", "Fuel type", "Gear box type", "Drive wheels", "Color"], dtype="int64")
df_dummy.head()

,Price,Levy,Prod. year,Leather interior,Engine volume,Mileage,Cylinders,Airbags,Turbo,Manufacturer_AUDI,...,Gear box type_Variator,Drive wheels_4x4,Drive wheels_Front,Drive wheels_Rear,Color_Black,Color_Blue,Color_Grey,Color_Other,Color_Silver,Color_White
0,13328,1399,2010,1,3.5,186005,6.0,12,0,0,...,0,1,0,0,0,0,0,0,1,0
1,16621,1018,2011,0,3.0,192000,6.0,8,0,0,...,0,1,0,0,1,0,0,0,0,0
2,8467,0,2006,0,1.3,200000,4.0,2,0,0,...,1,0,1,0,1,0,0,0,0,0
3,3607,862,2011,1,2.5,168966,4.0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,11726,446,2014,1,1.3,91901,4.0,4,0,0,...,0,0,1,0,0,0,0,0,1,0


### Creating the Model

In [38]:
# split preprocessed data into features and target arrays
y = df_dummy["Price"]
X = df_dummy.drop(["Price"], axis=1)

# split this further into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
# create a scaler for X
scaler = StandardScaler()

# fit the scaler
X_scaler = scaler.fit(X_train)

# scale the X data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# reshape y
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# create a scaler for y
y_scaler = scaler.fit(y_train)

# scale the y data
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [40]:
# save the X and y scalers for later use
# https://stackoverflow.com/questions/53152627/saving-standardscaler-model-for-use-on-new-datasets
dump(X_scaler, "../../data/X_scaler.bin", compress=True)
dump(y_scaler, "../../data/y_scaler.bin", compress=True)

['../../data/y_scaler.bin']

In [41]:
def create_model(hp):
    # create model
    nn = tf.keras.models.Sequential()

    # create list containing different activation functions
    activation = hp.Choice("activation", ["relu", "tanh"])

    # add first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int("first_units",
                                              min_value=16,
                                              max_value=256,
                                              step=32),
                                              activation=activation,
                                              input_dim=len(X.columns)))
    
    # add random amount of hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):
        nn.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
                                                  min_value=16,
                                                  max_value=128,
                                                  step=16),
                                                  activation=activation))
        
    # add output layer
    nn.add(tf.keras.layers.Dense(units=1))

    # compile model
    nn.compile(loss="mean_absolute_error", optimizer="adam")

    # return model
    return nn

In [42]:
# create tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_loss",
    max_epochs=100,
    project_name="smart_model_history"
)

# run tuner
tuner.search(X_train_scaled, y_train_scaled, epochs=100, validation_data=(X_test_scaled, y_test_scaled))

Reloading Tuner from .\smart_model_history\tuner0.json


In [43]:
# get the best model
nn = tuner.get_best_models(1)[0]

c:\Users\Paul\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Paul\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
# make predictions using test data
predictions = nn.predict(X_test_scaled)

133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step


In [45]:
# evaluate the model using test data
mean_abs_err = mean_absolute_error(y_test_scaled, predictions)
mean_sq_err = mean_squared_error(y_test_scaled, predictions)
r2 = r2_score(y_test_scaled, predictions)

print(f"Mean Absolute Error: {mean_abs_err}\nMean Squared Error: {mean_sq_err}\nR^2: {r2}")

Mean Absolute Error: 0.13379299778849854
Mean Squared Error: 0.09908105777366513
R^2: 0.8829478385867243


In [46]:
# predict the first test value using model
y_pred_scaled = nn.predict(X_test_scaled[0].reshape(1, -1))
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
print(f"Prediction: {y_pred_original[0][0]}")

# actual first test value
y_test_original = y_scaler.inverse_transform(y_test_scaled[0].reshape(1, -1))
print(f"Actual: {y_test_original[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Prediction: 18565.06640625
Actual: 18817.0


In [47]:
# save the model
nn.save("smart_model.keras")

In [48]:
nn.get_config()

{'name': 'sequential',
 'trainable': True,
 'dtype': {'module': 'keras',
  'class_name': 'DTypePolicy',
  'config': {'name': 'float32'},
  'registered_name': None},
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_shape': (None, 63),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_layer'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': {'module': 'keras',
     'class_name': 'DTypePolicy',
     'config': {'name': 'float32'},
     'registered_name': None},
    'units': 240,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regulariz